In [1]:
from qutip import *
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
from scipy.special import genlaguerre as L

In [8]:
import gmpy2

In [2]:
hbar = sp.constants.hbar
h = sp.constants.h
c = sp.constants.c
e = sp.constants.e
eVtoJ = 1.602176634e-19
amutokg = 1.66053906660e-27

In [11]:
gmpy2.set_context(gmpy2.ieee(128))
gmpy2.sin(gmpy2.mpfr("1.2"))
a = 1.0

In [3]:
def psi(g,e):
    return np.array([g, e])

def pulse(t, Omega0, delta, phi0, phi1):
    Omega = np.sqrt(Omega0**2 + delta**2)
    
    if Omega == 0.0:
        U = np.array([[1.0 , 0.0],
                      [0.0 , 1.0]])
        
    else:
        U = np.array([[np.cos(Omega*t/2.0) -(1.0j*delta/Omega)*np.sin(Omega*t/2.0), (1.0j*Omega0/Omega)*np.sin(Omega*t/2.0)],
                     [(1.0j*Omega0/Omega)*np.sin(Omega*t/2.0) , np.cos(Omega*t/2.0) + (1.0j*delta/Omega)*np.sin(Omega*t/2.0)]])
    
    V1 = np.array([[1.0 , 0.0],
                   [0.0 , np.exp(-1.0j*phi1)]])
    
    V0 = np.array([[1.0 , 0.0],
                   [0.0 , np.exp(1.0j*phi0)]])
    
    return np.exp(1.0j*delta*t/2.0)*V1@U@V0

def thermPulse(psi0, t, Omega0, delPrime, eta, nbar, Ncut): #1-dimensional
    result = psi(0.0+0.0j,0.0+0.0j)
    for n in range(Ncut):
        result += prob(n,nbar)*pulse(t, Omega0*Omega(n,0,eta), delPrime, 0.0, 0.0)@psi0
    return result

def therm3DPulse(psi0, t, Omega0, delPrime, nbars, Omegas, Ncut):
    nbarx = nbars[0]
    nbary = nbars[1]
    nbarz = nbars[2]
    #Ncut = 100
    result = psi(0.0+0.0j,0.0+0.0j)
    for nx in range(Ncut):
        Probx = prob(nx,nbarx)
        for ny in range(Ncut):
            Probxy = Probx*prob(ny,nbary)
            for nz in range(Ncut):
                result += Probxy*prob(nz,nbarz)*pulse(t, Omega0*Omegas[nx,ny,nz], delPrime, 0.0, 0.0)@psi0
    return result

def rhoee(psi):
    return (abs(psi[1]))**2.0

def rhogg(psi):
    return (abs(psi[0]))**2.0

def prob(n,nbar):
    return 1.0/(nbar+1)*(nbar/(nbar+1))**n    #returns prob of being in state n given nbar

def Omega(n,m,eta):
    return np.exp(-eta**2/2.0) * eta**(np.abs(m)) * (math.factorial(n)/math.factorial(n+m))**(np.sign(m)/2.0) \
        * L(n,np.abs(m))(eta**2)              #returns unitless rabi rate of transition n -> n+m given Lamb-Dicke Parameter

In [4]:
def heatPulse(psi0, t, Omega0, delPrime, eta, nbar0, nbarf, Ncut, tsteps):
    dt = t/tsteps
    nbars = np.linspace(nbar0, nbarf, tsteps)
    state = psi0
    for ii in range(tsteps):
        state = thermPulse(state, dt, Omega0, delPrime, eta, nbars[ii], Ncut)
    return state

In [41]:
psi0 = psi(1.0,0.0)
t = 20e-3
Omega0 = np.pi/(2.0*t)
delPrime = 0.0
eta = 0.05
nbar = 1.0
#nbarf = 10.0
Ncut = 700
tsteps = 2

heatpsi = heatPulse(psi0, t, Omega0, delPrime, eta, nbar, nbar, Ncut, tsteps)
psi1 = thermPulse(psi0, t, Omega0, delPrime, eta, nbar, Ncut)

In [44]:
rhoee(heatpsi)

0.4970583891150105

In [45]:
rhoee(psi1)

0.497056492714418

In [9]:
for ii in range(1):
    print(ii)

0


In [19]:
ts = np.linspace(0.0, 20e-3, 1+1)

In [20]:
ts

array([0.  , 0.02])

In [79]:
t/tsteps*tsteps

0.02

In [21]:
nbar0 = 10
nbarf = 15
tsteps=1
nbars = np.linspace(nbar0, nbarf, tsteps)

In [22]:
nbars

array([10.])

In [78]:
for ii in range(tsteps):
    print(nbars[ii])

10.0
10.0
10.0
10.0
10.0


In [12]:
a

1.0

In [32]:
prob(1,1.0)

0.25

In [34]:
prob(3,1.0)


0.0625